# This notebook examines horizontal regridding of ERA5 topography onto CAM grids

In [ ]:
# Import packages 
import sys
sys.path.append('../Plotting/')
""" Now you can imprt modules in ../Plotting"""



import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import ana as a

import ESMF as E

import importlib

import copy
import glob


import xyp_plot as xyp
import esmfRegrid as erg


importlib.reload( erg )

## Read in scrip files

In [ ]:
scripdir='/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'

ne30scrip  = scripdir +  "ne30pg3_scrip_170611.nc"
fv1x1scrip = scripdir +  "fv0.9x1.25_141008.nc"

# Scrip file for ERA5 created by ERA5scrip.ipynb
ERA5scrip = '/glade/work/juliob/ERA5-proc/ERA5interp/grids/ERA5_640x1280_scrip.nc'

## Next few cells look at some content of scrip files, topo data etc.

In [ ]:
ne30x = xr.open_dataset( ne30scrip )
print( list( ne30x.variables ) )

eratopof='/glade/work/juliob/ERA5-proc/ERA5interp/phis/ERA5_phis.nc'
eratopo=xr.open_dataset( eratopof )
Hera=eratopo['Z_GDS4_SFC'].values
camtopof='/glade/work/juliob/ERA5-proc/ERA5interp/phis/CAM6_phis.nc'
camtopo=xr.open_dataset( camtopof )
Hcam=camtopo['PHIS'].values

In [ ]:
print( list( camtopo.variables ) )

In [ ]:
plt.contourf( Hera)

## Set up inputs for call to erg.Regrid (in esmfRegrid.py)

In [ ]:
src_scrip=ERA5scrip
src_type='grid'
# ------- CAM SE ne30
dst_scrip=ne30scrip
dst_type='mesh'
# ------- CAM FV 1 degree
#dst_scrip=fv1x1scrip
#dst_type='grid'

## Call regridding: 1st conservative (default), then bi-linear

In [ ]:
griddir = "/glade/work/juliob/ERA5-proc/ERA5interp/grids/"

wgts_file_Con = griddir + "ERA5_ne30pg3_Conserv_wgts.nc"
wgts_file_BiL = griddir + "ERA5_ne30pg3_BiLinear_wgts.nc"

In [ ]:
# Conservative regridding
regrd, srcf, dstf = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                write_weights = False ,
                                read_weights = True ,
                                weights_file = wgts_file_Con )

# Bi-linear regridding
regrdL, srcfL, dstfL = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                RegridMethod='BILINEAR',
                                write_weights = False ,
                                read_weights = True ,
                                weights_file = wgts_file_BiL )

In [ ]:
print(np.shape(srcf.data))
print(np.shape(Hera.transpose()))


In [ ]:
srcf.data[:,:]  = Hera.transpose()
srcfL.data[:,:] = Hera.transpose()



In [ ]:
r  = regrd(  srcf,  dstf )
rL = regrdL( srcfL, dstfL )

In [ ]:
ne30 = xr.open_dataset( ne30scrip )
fv1  = xr.open_dataset( fv1x1scrip )
era  = xr.open_dataset( ERA5scrip )

### Get lat lons for model and ERA5. Note, we can use scrip file centers for SE ne30

In [ ]:
lon=ne30['grid_center_lon']
lat=ne30['grid_center_lat']
#lon=camtopo['lon']
#lat=camtopo['lat']
elat=eratopo['g4_lat_2'].values
elon=eratopo['g4_lon_3'].values


# Plot regridded topography and raw ERA5 topo

In [ ]:
clev=np.linspace( 5, 5600, num=31 )

fig,ax=plt.subplots(3  , figsize=(10,30) )

po=ax[0].tricontourf(lon,lat,dstf.data[:] / 9.8  ,levels=clev )
#po=ax[0].contourf(lon,lat,dstf.data[:,:].transpose() / 9.8  ,levels=clev )
plt.colorbar( po , ax=ax[0] ) #po,cax=ax)

po=ax[1].tricontourf(lon,lat,dstfL.data[:] / 9.8  ,levels=clev )
#po=ax[1].contourf(lon,lat,dstfL.data[:,:].transpose() / 9.8  ,levels=clev )
plt.colorbar( po , ax=ax[1] ) #po,cax=ax)

po=ax[2].contourf(elon,elat, Hera/ 9.8  ,levels=clev  )
plt.colorbar( po , ax=ax[2] ) #po,cax=ax)

ax[0].set_title('Conservatively Regridded ERA5 Topo (m) on ne30pg3 grid')
ax[1].set_title('Bilinearly Regridded ERA5 Topo (m) on ne30pg3 grid')
ax[2].set_title('Topo height in ERA5')


## Plot differences bewteen conservative and bi-linear regridding

In [ ]:
diffCL = dstf.data- dstfL.data
plt.plot(diffCL.flatten()/9.8)

In [ ]:
clev=np.linspace( 5, 5600, num=31 )
dlev=np.linspace( -600, 600, num=31 )

fig,ax=plt.subplots(3  , figsize=(10,30) )

po=ax[0].tricontourf(lon,lat,dstf.data[:] / 9.8  ,levels=clev )
plt.colorbar( po , ax=ax[0] ) #po,cax=ax)

po=ax[1].tricontourf(lon,lat,dstfL.data[:] / 9.8  ,levels=clev )
plt.colorbar( po , ax=ax[1] ) #po,cax=ax)

diffCL = dstf.data[:] - dstfL.data[:]
po=ax[2].tricontourf(lon, lat, diffCL/ 9.8  ,levels=dlev  )
plt.colorbar( po , ax=ax[2] ) #po,cax=ax)


ax[0].set_title('Cons. Regridded ERA5 Topo (m) on ne30pg3 grid')
ax[1].set_title('BiLin. Regridded ERA5 Topo (m) on ne30pg3 grid')
ax[2].set_title('Difference (Cons-BiLin)')
